#### 對應期數

1. load TCGA barcode data and sample sheet
2. corresponding barcode to TCGA barcode
3. create dataframe to store result
4. map each barcode to corresponding Sentrix_ID and Sentrix_Position
5. load phenotype information
6. map to corresponding Sample_Group, Sample_Name, stage, type and age
7. save result to csv file

In [11]:
import pandas as pd

input_path = "../breast"

# Comment [TCGA Barcode] | Array Data File
TCGA_barcode = pd.read_csv(
    f"{input_path}/external_result/jhu-usc.edu_BRCA.HumanMethylation450.1.11.0.sdrf.txt.csv"
)

TCGA_barcode

,Comment [TCGA Barcode],Array Data File
0,TCGA-A7-A0DC-01B-04D-A22R-05,7796806125_R03C02_Grn.idat
1,TCGA-A7-A0DC-01B-04D-A22R-05,7796806125_R03C02_Red.idat
2,TCGA-A7-A0CE-11A-21D-A10Q-05,6005486013_R02C02_Grn.idat
3,TCGA-A7-A0CE-11A-21D-A10Q-05,6005486013_R02C02_Red.idat
4,TCGA-BH-A0BV-11A-31D-A10Q-05,6005486013_R03C02_Grn.idat
...,...,...
1855,TCGA-PE-A5DC-01A-12D-A27Y-05,9298768172_R02C02_Red.idat
1856,TCGA-PE-A5DD-01A-12D-A27Y-05,9298768134_R06C01_Grn.idat
1857,TCGA-PE-A5DD-01A-12D-A27Y-05,9298768134_R06C01_Red.idat
1858,TCGA-PE-A5DE-01A-11D-A27Y-05,9298768134_R02C02_Grn.idat


In [18]:
# Sample_Name | Sample_Plate | Sample_Group | Pool_ID | Sentrix_ID | Sentrix_Position
samplesheet = pd.read_csv(f"{input_path}/data/GDC_breast_tissue/sample_sheet_0.csv")
samplesheet

,Sample_Name,Sample_Plate,Sample_Group,Pool_ID,Sentrix_ID,Sentrix_Position
0,1,NaN,Normal,NaN,6026818108,R03C02
1,2,NaN,Normal,NaN,6026818108,R03C02
2,3,NaN,Normal,NaN,6026818135,R04C02
3,4,NaN,Normal,NaN,6026818135,R04C02
4,5,NaN,Normal,NaN,6026818144,R05C02
...,...,...,...,...,...,...
885,886,NaN,Tumor,NaN,9298768172,R02C02
886,887,NaN,Tumor,NaN,9298768134,R06C01
887,888,NaN,Tumor,NaN,9298768134,R06C01
888,889,NaN,Tumor,NaN,9298768134,R02C02


#### 對應 Sentrix_ID | Sentrix_Position | barcode	

In [19]:
result = pd.DataFrame(
    columns=[
        "Sample_Name",
        "Sentrix_ID",
        "Sentrix_Position",
        "barcode",
        "Sample_Group",
        "stage",
        "type",
    ]
)

for j in range(0, len(samplesheet)):

    for i in range(0, len(TCGA_barcode)):
        ID = TCGA_barcode["Array Data File"][i][:10]  # 7796806125
        Position = TCGA_barcode["Array Data File"][i][11:17]  # R03C02
        barcode = TCGA_barcode["Comment [TCGA Barcode]"][i]  # TCGA-A7-A0DC-01B-04D-A22R-05


        if (str(ID) == str(samplesheet["Sentrix_ID"][j])) and (
            str(Position) == str(samplesheet["Sentrix_Position"][j])):
            new_row = pd.DataFrame(
                {
                    "Sentrix_ID": [samplesheet["Sentrix_ID"][j]],
                    "Sentrix_Position": [samplesheet["Sentrix_Position"][j]],
                    "barcode": [barcode],
                }
            )
            result = pd.concat([result, new_row], ignore_index=True)
            break
result

,Sample_Name,Sentrix_ID,Sentrix_Position,barcode,Sample_Group,stage,type
0,NaN,6026818108,R03C02,TCGA-BH-A1FN-11A-34D-A13T-05,NaN,NaN,NaN
1,NaN,6026818108,R03C02,TCGA-BH-A1FN-11A-34D-A13T-05,NaN,NaN,NaN
2,NaN,6026818135,R04C02,TCGA-A7-A13G-11A-51D-A13T-05,NaN,NaN,NaN
3,NaN,6026818135,R04C02,TCGA-A7-A13G-11A-51D-A13T-05,NaN,NaN,NaN
4,NaN,6026818144,R05C02,TCGA-BH-A1F6-11B-94D-A13T-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...
885,NaN,9298768172,R02C02,TCGA-PE-A5DC-01A-12D-A27Y-05,NaN,NaN,NaN
886,NaN,9298768134,R06C01,TCGA-PE-A5DD-01A-12D-A27Y-05,NaN,NaN,NaN
887,NaN,9298768134,R06C01,TCGA-PE-A5DD-01A-12D-A27Y-05,NaN,NaN,NaN
888,NaN,9298768134,R02C02,TCGA-PE-A5DE-01A-11D-A27Y-05,NaN,NaN,NaN


#### 對應Sample_Name | Sample_Group | stage | type

* Sample_Name: 樣本編號
* Sample_Group: Normal/Tumor
* stage: 期數 (Normal留NaN)
* type: Normal/Tumor更細分類別 ex: Solid Tissue Normal、primary tumor等

In [14]:
# 有期數的檔案
phenotype = pd.read_csv(
    f"{input_path}/external_result/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix.csv"
)
phenotype

,sampleID,AJCC_Stage_nature2012,Age_at_Initial_Pathologic_Diagnosis_nature2012,CN_Clusters_nature2012,Converted_Stage_nature2012,Days_to_Date_of_Last_Contact_nature2012,Days_to_date_of_Death_nature2012,ER_Status_nature2012,Gender_nature2012,HER2_Final_Status_nature2012,...,_GENOMIC_ID_TCGA_BRCA_mutation_wustl_gene,_GENOMIC_ID_TCGA_BRCA_miRNA_GA,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2_percentile,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_GA_gene,_GENOMIC_ID_TCGA_BRCA_gistic2thd,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_HiSeq_gene,_GENOMIC_ID_TCGA_BRCA_G4502A_07_3,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2,_GENOMIC_ID_TCGA_BRCA_gistic2,_GENOMIC_ID_TCGA_BRCA_PDMarray
0,TCGA-3C-AAAU-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6ef883fc-81f3-4089-95e0-86904ffc0d38,NaN,TCGA-3C-AAAU-01A-11D-A41E-01,TCGA-3C-AAAU-01,NaN,6ef883fc-81f3-4089-95e0-86904ffc0d38,TCGA-3C-AAAU-01A-11D-A41E-01,NaN
1,TCGA-3C-AALI-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,dd8d3665-ec9d-45be-b7b9-a85dac3585e2,NaN,TCGA-3C-AALI-01A-11D-A41E-01,TCGA-3C-AALI-01,NaN,dd8d3665-ec9d-45be-b7b9-a85dac3585e2,TCGA-3C-AALI-01A-11D-A41E-01,NaN
2,TCGA-3C-AALJ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,c924c2a8-ab41-4499-bb30-79705cc17d45,NaN,TCGA-3C-AALJ-01A-31D-A41E-01,TCGA-3C-AALJ-01,NaN,c924c2a8-ab41-4499-bb30-79705cc17d45,TCGA-3C-AALJ-01A-31D-A41E-01,NaN
3,TCGA-3C-AALK-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1a19e068-d213-41ef-aebb-104017a883cc,NaN,TCGA-3C-AALK-01A-11D-A41E-01,TCGA-3C-AALK-01,NaN,1a19e068-d213-41ef-aebb-104017a883cc,TCGA-3C-AALK-01A-11D-A41E-01,NaN
4,TCGA-4H-AAAK-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2ea9e472-a408-4ae0-975d-50a566f22b2a,NaN,TCGA-4H-AAAK-01A-12D-A41E-01,TCGA-4H-AAAK-01,NaN,2ea9e472-a408-4ae0-975d-50a566f22b2a,TCGA-4H-AAAK-01A-12D-A41E-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,TCGA-XX-A899-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8e339298-3e3c-4b31-96a9-a842e973e583,NaN,TCGA-XX-A899-01A-11D-A36I-01,TCGA-XX-A899-01,NaN,8e339298-3e3c-4b31-96a9-a842e973e583,TCGA-XX-A899-01A-11D-A36I-01,NaN
1243,TCGA-XX-A899-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1244,TCGA-XX-A89A-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,501a70d8-3932-49c0-aa13-774756fb89e3,NaN,TCGA-XX-A89A-01A-11D-A36I-01,TCGA-XX-A89A-01,NaN,501a70d8-3932-49c0-aa13-774756fb89e3,TCGA-XX-A89A-01A-11D-A36I-01,NaN
1245,TCGA-Z7-A8R5-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,43c6d23b-2fcd-499e-82ef-10f15d8ede90,NaN,TCGA-Z7-A8R5-01A-42D-A41E-01,TCGA-Z7-A8R5-01,NaN,43c6d23b-2fcd-499e-82ef-10f15d8ede90,TCGA-Z7-A8R5-01A-42D-A41E-01,NaN


In [20]:
for j in range(0, len(result)):
    for i in range(0, len(phenotype["_GENOMIC_ID_TCGA_BRCA_hMethyl450"])):

        if str(result["barcode"][j]) == str(phenotype["_GENOMIC_ID_TCGA_BRCA_hMethyl450"][i]):  # 找到barcode
            stage = phenotype["AJCC_Stage_nature2012"][i]
            type = phenotype["sample_type"][i]
            result.loc[j, "stage"] = stage
            result.loc[j, "type"] = type
            break

result["Sample_Group"] = samplesheet["Sample_Group"]
result["Sample_Name"] = samplesheet["Sample_Name"]
result

,Sample_Name,Sentrix_ID,Sentrix_Position,barcode,Sample_Group,stage,type
0,1,6026818108,R03C02,TCGA-BH-A1FN-11A-34D-A13T-05,Normal,NaN,Solid Tissue Normal
1,2,6026818108,R03C02,TCGA-BH-A1FN-11A-34D-A13T-05,Normal,NaN,Solid Tissue Normal
2,3,6026818135,R04C02,TCGA-A7-A13G-11A-51D-A13T-05,Normal,NaN,Solid Tissue Normal
3,4,6026818135,R04C02,TCGA-A7-A13G-11A-51D-A13T-05,Normal,NaN,Solid Tissue Normal
4,5,6026818144,R05C02,TCGA-BH-A1F6-11B-94D-A13T-05,Normal,NaN,Solid Tissue Normal
...,...,...,...,...,...,...,...
885,886,9298768172,R02C02,TCGA-PE-A5DC-01A-12D-A27Y-05,Tumor,NaN,Primary Tumor
886,887,9298768134,R06C01,TCGA-PE-A5DD-01A-12D-A27Y-05,Tumor,NaN,Primary Tumor
887,888,9298768134,R06C01,TCGA-PE-A5DD-01A-12D-A27Y-05,Tumor,NaN,Primary Tumor
888,889,9298768134,R02C02,TCGA-PE-A5DE-01A-11D-A27Y-05,Tumor,NaN,Primary Tumor


#### 對應age

In [21]:
for j in range(0, len(result)):
    for i in range(0, len(phenotype["_GENOMIC_ID_TCGA_BRCA_hMethyl450"])):

        if str(result["barcode"][j]) == str(
            phenotype["_GENOMIC_ID_TCGA_BRCA_hMethyl450"][i]
        ):
            age = phenotype["age_at_initial_pathologic_diagnosis"][i]
            if age >= 0:
                result.loc[j, "age"] = int(age)
                break
result

,Sample_Name,Sentrix_ID,Sentrix_Position,barcode,Sample_Group,stage,type,age
0,1,6026818108,R03C02,TCGA-BH-A1FN-11A-34D-A13T-05,Normal,NaN,Solid Tissue Normal,34.0
1,2,6026818108,R03C02,TCGA-BH-A1FN-11A-34D-A13T-05,Normal,NaN,Solid Tissue Normal,34.0
2,3,6026818135,R04C02,TCGA-A7-A13G-11A-51D-A13T-05,Normal,NaN,Solid Tissue Normal,79.0
3,4,6026818135,R04C02,TCGA-A7-A13G-11A-51D-A13T-05,Normal,NaN,Solid Tissue Normal,79.0
4,5,6026818144,R05C02,TCGA-BH-A1F6-11B-94D-A13T-05,Normal,NaN,Solid Tissue Normal,51.0
...,...,...,...,...,...,...,...,...
885,886,9298768172,R02C02,TCGA-PE-A5DC-01A-12D-A27Y-05,Tumor,NaN,Primary Tumor,72.0
886,887,9298768134,R06C01,TCGA-PE-A5DD-01A-12D-A27Y-05,Tumor,NaN,Primary Tumor,64.0
887,888,9298768134,R06C01,TCGA-PE-A5DD-01A-12D-A27Y-05,Tumor,NaN,Primary Tumor,64.0
888,889,9298768134,R02C02,TCGA-PE-A5DE-01A-11D-A27Y-05,Tumor,NaN,Primary Tumor,41.0


In [22]:
result.to_csv(f"{input_path}/result/GDC_breast_tissue_450k/stage/stage_0.csv", index=False)